# Project: Bollinger Bands Mean Reversion Strategy

When prices move far from their average (more than a few standard deviations), they tend to revert.
We’ll go long when the price dips below the lower Bollinger Band (oversold) and short when it rises above the upper band (overbought).

We’ll only take Bollinger Band mean reversion signals when the RSI confirms oversold or overbought conditions — improving signal quality and avoiding “catching falling knives.”

## Imports

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

%load_ext autoreload
%autoreload 2

## Data Setup

In [2]:
ticker = 'AAPL'
start_date = '2015-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')

ticker, start_date, end_date

('AAPL', '2015-01-01', '2025-10-23')

In [36]:
df = yf.download(ticker, start=start_date, end=end_date)
df = df.droplevel(axis=1, level=1)['Close'].to_frame()
df.dropna(inplace=True)
df.head()

[*********************100%***********************]  1 of 1 completed


,Close
Date,
2015-01-02,24.261055
2015-01-05,23.577572
2015-01-06,23.579792
2015-01-07,23.910435
2015-01-08,24.829128


## Set Indicators

In [37]:
window = 20
num_std = 2
rsi_period = 14

In [38]:
df

,Close
Date,
2015-01-02,24.261055
2015-01-05,23.577572
2015-01-06,23.579792
2015-01-07,23.910435
2015-01-08,24.829128
...,...
2025-10-16,247.449997
2025-10-17,252.289993
2025-10-20,262.239990


In [39]:
df['MA'] = df['Close'].rolling(window).mean()
df['STD'] = df['Close'].rolling(window).std()
df['Upper'] = df['MA'] + num_std * df['STD']
df['Lower'] = df['MA'] - num_std * df['STD']

In [40]:
df

,Close,MA,STD,Upper,Lower
Date,,,,,
2015-01-02,24.261055,NaN,NaN,NaN,NaN
2015-01-05,23.577572,NaN,NaN,NaN,NaN
2015-01-06,23.579792,NaN,NaN,NaN,NaN
2015-01-07,23.910435,NaN,NaN,NaN,NaN
2015-01-08,24.829128,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2025-10-16,247.449997,253.153499,4.316475,261.786450,244.520548
2025-10-17,252.289993,253.492999,3.932802,261.358602,245.627395
2025-10-20,262.239990,253.800999,4.363676,262.528351,245.073647


In [25]:
df['delta'] = df['Close'].diff()

In [26]:
df

,Close,MA,STD,Upper,Lower,delta
Date,,,,,,
2015-01-02,24.261055,NaN,NaN,NaN,NaN,NaN
2015-01-05,23.577572,NaN,NaN,NaN,NaN,-0.683483
2015-01-06,23.579792,NaN,NaN,NaN,NaN,0.002220
2015-01-07,23.910435,NaN,NaN,NaN,NaN,0.330643
2015-01-08,24.829128,NaN,NaN,NaN,NaN,0.918694
...,...,...,...,...,...,...
2025-10-16,247.449997,253.153499,4.316475,261.786450,244.520548,-1.889999
2025-10-17,252.289993,253.492999,3.932802,261.358602,245.627395,4.839996
2025-10-20,262.239990,253.800999,4.363676,262.528351,245.073647,9.949997


In [41]:
# RSI Calculation

delta = df.Close.diff().copy()
gain = np.where(delta > 0, delta, 0)
loss = np.where(delta < 0, -delta, 0)
avg_gain = pd.Series(gain).rolling(rsi_period).mean()
avg_loss = pd.Series(loss).rolling(rsi_period).mean()
rs = avg_gain / avg_loss

df['RSI'] = pd.Series((100 - (100 / (1 + rs))))
#df.dropna(inplace=True)

In [64]:
# RSI Calculation

delta = df.Close.diff()
gain = np.where(delta > 0, delta, 0)
loss = np.where(delta < 0, -delta, 0)
avg_gain = pd.Series(gain).rolling(rsi_period).mean()
avg_loss = pd.Series(loss).rolling(rsi_period).mean()
rs = (avg_gain / avg_loss).values
df['RSI'] = 100 - (100 / (1 + rs))

df.dropna(inplace=True)
df


,Close,MA,STD,Upper,Lower,RSI
Date,,,,,,
2015-01-30,25.998579,24.552079,0.797320,26.146719,22.957439,58.434299
2015-02-02,26.324776,24.655265,0.886254,26.427773,22.882758,66.039687
2015-02-03,26.329218,24.792847,0.922970,26.638787,22.946908,64.899260
2015-02-04,26.531151,24.940415,0.954222,26.848860,23.031971,66.956193
2015-02-05,26.720518,25.080920,1.000351,27.081621,23.080218,75.500310
...,...,...,...,...,...,...
2025-10-16,247.449997,253.153499,4.316475,261.786450,244.520548,34.881046
2025-10-17,252.289993,253.492999,3.932802,261.358602,245.627395,46.468641
2025-10-20,262.239990,253.800999,4.363676,262.528351,245.073647,59.500623


## Generate Trading Signals

Conditions:
* Long when price < lower band AND RSI < 30 (oversold)
* Short when price > upper band AND RSI > 70 (overbought)
* Exit when price crosses back towards MA

In [65]:
df['Signal'] = 0
df

,Close,MA,STD,Upper,Lower,RSI,Signal
Date,,,,,,,
2015-01-30,25.998579,24.552079,0.797320,26.146719,22.957439,58.434299,0
2015-02-02,26.324776,24.655265,0.886254,26.427773,22.882758,66.039687,0
2015-02-03,26.329218,24.792847,0.922970,26.638787,22.946908,64.899260,0
2015-02-04,26.531151,24.940415,0.954222,26.848860,23.031971,66.956193,0
2015-02-05,26.720518,25.080920,1.000351,27.081621,23.080218,75.500310,0
...,...,...,...,...,...,...,...
2025-10-16,247.449997,253.153499,4.316475,261.786450,244.520548,34.881046,0
2025-10-17,252.289993,253.492999,3.932802,261.358602,245.627395,46.468641,0
2025-10-20,262.239990,253.800999,4.363676,262.528351,245.073647,59.500623,0


In [66]:
df.loc[(df["Close"] < df["Lower"]) & (df["RSI"] < 30), "Signal"] = 1
df.loc[(df["Close"] > df["Upper"]) & (df["RSI"] > 70), "Signal"] = -1
df

,Close,MA,STD,Upper,Lower,RSI,Signal
Date,,,,,,,
2015-01-30,25.998579,24.552079,0.797320,26.146719,22.957439,58.434299,0
2015-02-02,26.324776,24.655265,0.886254,26.427773,22.882758,66.039687,0
2015-02-03,26.329218,24.792847,0.922970,26.638787,22.946908,64.899260,0
2015-02-04,26.531151,24.940415,0.954222,26.848860,23.031971,66.956193,0
2015-02-05,26.720518,25.080920,1.000351,27.081621,23.080218,75.500310,0
...,...,...,...,...,...,...,...
2025-10-16,247.449997,253.153499,4.316475,261.786450,244.520548,34.881046,0
2025-10-17,252.289993,253.492999,3.932802,261.358602,245.627395,46.468641,0
2025-10-20,262.239990,253.800999,4.363676,262.528351,245.073647,59.500623,0


In [68]:
df.loc[df["Close"].between(df["Lower"], df["Upper"]), "Signal"] = 0
df


,Close,MA,STD,Upper,Lower,RSI,Signal
Date,,,,,,,
2015-01-30,25.998579,24.552079,0.797320,26.146719,22.957439,58.434299,0
2015-02-02,26.324776,24.655265,0.886254,26.427773,22.882758,66.039687,0
2015-02-03,26.329218,24.792847,0.922970,26.638787,22.946908,64.899260,0
2015-02-04,26.531151,24.940415,0.954222,26.848860,23.031971,66.956193,0
2015-02-05,26.720518,25.080920,1.000351,27.081621,23.080218,75.500310,0
...,...,...,...,...,...,...,...
2025-10-16,247.449997,253.153499,4.316475,261.786450,244.520548,34.881046,0
2025-10-17,252.289993,253.492999,3.932802,261.358602,245.627395,46.468641,0
2025-10-20,262.239990,253.800999,4.363676,262.528351,245.073647,59.500623,0


In [72]:
df['Position'] = df['Signal'].shift(1)

df.dropna(inplace=True)

df

,Close,MA,STD,Upper,Lower,RSI,Signal,Position
Date,,,,,,,,
2015-02-03,26.329218,24.792847,0.922970,26.638787,22.946908,64.899260,0,0.0
2015-02-04,26.531151,24.940415,0.954222,26.848860,23.031971,66.956193,0,0.0
2015-02-05,26.720518,25.080920,1.000351,27.081621,23.080218,75.500310,0,0.0
2015-02-06,26.495499,25.164238,1.046602,27.257442,23.071035,74.982577,0,0.0
2015-02-09,26.671503,25.255025,1.096021,27.447067,23.062983,73.036603,0,0.0
...,...,...,...,...,...,...,...,...
2025-10-16,247.449997,253.153499,4.316475,261.786450,244.520548,34.881046,0,0.0
2025-10-17,252.289993,253.492999,3.932802,261.358602,245.627395,46.468641,0,0.0
2025-10-20,262.239990,253.800999,4.363676,262.528351,245.073647,59.500623,0,0.0
